In [ ]:
import random
import time
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import benchmarks
import arff


In [ ]:
# Load the data in arff file as list
data = arff.load("Datasets/Colon.arff")

# Convert the list into dataframe
data = pd.DataFrame(data)

In [ ]:
X = data.drop(2000, axis=1)
Y = data[2000]

print(X.shape)

# Applying SVM classification without data processing

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 14)

''' 
sigmoid => 0.68
rbf     => 0.88
poly    => 0.72
linear  => 0.72
'''

svc_classifier = SVC(kernel="rbf")
svc_classifier.fit(X_train, y_train)
y_pred = svc_classifier.predict(X_test)
P = accuracy_score(y_pred, y_test)
print("Accuracy score for SVM:", P)

In [ ]:
def GWO(objf,lb,ub,dim,SearchAgents_no,Max_iter):
    
    # initialize alpha, beta, and delta_pos
    Alpha_pos=np.zeros(dim)
    Alpha_score=float("inf")
    
    Beta_pos=np.zeros(dim)
    Beta_score=float("inf")
    
    Delta_pos=np.zeros(dim)
    Delta_score=float("inf")

    if not isinstance(lb, list):
        lb = [lb] * dim
    if not isinstance(ub, list):
        ub = [ub] * dim
    
    #Initialize the positions of search agents
    Positions = np.zeros((SearchAgents_no, dim))
    for i in range(dim):
        Positions[:, i] = np.random.uniform(0,1, SearchAgents_no) * (ub[i] - lb[i]) + lb[i]
    
    Convergence_curve=np.zeros(Max_iter)

     # Loop counter
    # print("GWO is optimizing  \""+objf.__name__+"\"")    

    # Main loop
    for l in range(0,Max_iter):
        for i in range(0,SearchAgents_no):
            
            # Return back the search agents that go beyond the boundaries of the search space
            for j in range(dim):
                Positions[i,j]=np.clip(Positions[i,j], lb[j], ub[j])

            # Calculate objective function for each search agent
            fitness=objf(Positions[i,:])
            
            # Update Alpha, Beta, and Delta
            if fitness<Alpha_score :
                Alpha_score=fitness; # Update alpha
                Alpha_pos=Positions[i,:].copy()
            
            
            if (fitness>Alpha_score and fitness<Beta_score ):
                Beta_score=fitness  # Update beta
                Beta_pos=Positions[i,:].copy()
            
            
            if (fitness>Alpha_score and fitness>Beta_score and fitness<Delta_score): 
                Delta_score=fitness # Update delta
                Delta_pos=Positions[i,:].copy()
        
        a=2-l*((2)/Max_iter); # a decreases linearly fron 2 to 0
        
        # Update the Position of search agents including omegas
        for i in range(0,SearchAgents_no):
            for j in range (0,dim):     
                           
                r1=random.random() # r1 is a random number in [0,1]
                r2=random.random() # r2 is a random number in [0,1]
                
                A1=2*a*r1-a; # Equation (3.3)
                C1=2*r2; # Equation (3.4)
                
                D_alpha=abs(C1*Alpha_pos[j]-Positions[i,j]); # Equation (3.5)-part 1
                X1=Alpha_pos[j]-A1*D_alpha; # Equation (3.6)-part 1
                           
                r1=random.random()
                r2=random.random()
                
                A2=2*a*r1-a; # Equation (3.3)
                C2=2*r2; # Equation (3.4)
                
                D_beta=abs(C2*Beta_pos[j]-Positions[i,j]); # Equation (3.5)-part 2
                X2=Beta_pos[j]-A2*D_beta; # Equation (3.6)-part 2       
                
                r1=random.random()
                r2=random.random() 
                
                A3=2*a*r1-a; # Equation (3.3)
                C3=2*r2; # Equation (3.4)
                
                D_delta=abs(C3*Delta_pos[j]-Positions[i,j]); # Equation (3.5)-part 3
                X3=Delta_pos[j]-A3*D_delta; # Equation (3.5)-part 3             
                
                Positions[i,j]=(X1+X2+X3)/3  # Equation (3.7)        
        Convergence_curve[l]=Alpha_score;

        #if (l%1==0):
               #print(['At iteration '+ str(l)+ ' the best fitness is '+ str(Alpha_score)]);
    
    # print(Positions.shape)
    # print("Alpha position=",Alpha_pos);
    # print("Beta position=",Beta_pos);
    # print("Delta position=",Delta_pos);
    return Alpha_pos,Beta_pos;

In [ ]:

for i in range(500):
    dimension = i+1

    print(f"dimension= {dimension}")

    iters=100
    wolves=10
    # dimension=50
    search_domain=[0,1]
    lb=-1.28
    ub=1.28
    colneeded=[0,1,2,4,5,7,8,10,11]
    modified_data=pd.DataFrame()
    for i in colneeded:
        modified_data[data.columns[i]]=data[data.columns[i]].astype(float)
    func_details=benchmarks.getFunctionDetails(6)

    for i in range(0,10):
        alpha,beta=GWO(getattr(benchmarks,'F7'),lb,ub,dimension,wolves,iters)

    threshold=-0.05
    index=[]
    # print("alpha shape=",alpha.shape[0])
    modified_daata=pd.DataFrame();
    for i in range(0,alpha.shape[0]):
        if(alpha[i]>=threshold):
            modified_daata[data.columns[i]]=data[data.columns[i]].astype(float)

    Y = data[2000]
    X_train, X_test, y_train, y_test = train_test_split(modified_daata, Y, test_size = 0.30, random_state = 14)

    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # print(X_train[0])

    print(f"shape: {modified_daata.shape}")

    svcclassifier = SVC(kernel = 'rbf')
    svcclassifier.fit(X_train, y_train)
    y_pred = svcclassifier.predict(X_test)
    P1=accuracy_score(y_pred,y_test)

    if P1 > P:
        print("Modified Accuracy is greater")
    if P1 == P:
        print("Modified Accuracy is equal")
    if P1 < P:
        print("Modified Accuracy is less")

    print("Modified Accuracy score for SVM:", P1)
    print("Earlier Accuracy score was", P)

In [ ]:
iters=100
wolves=5
dimension=50
search_domain=[0,1]
lb=-1.28
ub=1.28
colneeded=[0,1,2,4,5,7,8,10,11]
modified_data=pd.DataFrame()
for i in colneeded:
    modified_data[data.columns[i]]=data[data.columns[i]].astype(float)
func_details=benchmarks.getFunctionDetails(6)

for i in range(0,10):
    alpha,beta=GWO(getattr(benchmarks,'F7'),lb,ub,dimension,wolves,iters)

In [ ]:
##Applying feature selection on the given dataset
##considering alpha as best solution and putting a threshold
threshold=-0.05
index=[]
# print("alpha shape=",alpha.shape[0])
modified_daata=pd.DataFrame();
for i in range(0,alpha.shape[0]):
    if(alpha[i]>=threshold):
        modified_daata[data.columns[i]]=data[data.columns[i]].astype(float)
# print("The modified data is following")
# modified_daata.head()

In [ ]:
Y = data[2000]
X_train, X_test, y_train, y_test = train_test_split(modified_daata, Y, test_size = 0.30, random_state = 14)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
# print(X_train[0])
modified_daata.shape

In [ ]:
svcclassifier = SVC(kernel = 'rbf')
svcclassifier.fit(X_train, y_train)
y_pred = svcclassifier.predict(X_test)
P1=accuracy_score(y_pred,y_test)
print("Modified Accuracy score for SVM:", P1)
print("Earlier Accuracy score was", P)